<a href="https://colab.research.google.com/github/yongjulee0213/KLUE/blob/main/Copy_of_goorm_proj3_filtering_spoken_written_b_%EA%B3%B5%EC%9C%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan 27 22:36:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
os.chdir('/content/drive/MyDrive/NLPwChoo_project/proj3_mt')

In [ ]:
!pip install datasets konlpy sentence_transformers transformers
#!pip install git+https://github.com/ssut/py-hanspell.git
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6

In [ ]:
from datasets import Dataset
from IPython.display import display
from konlpy.tag import Okt
#from hanspell import spell_checker
#from pykospacing import Spacing
from nltk.translate.bleu_score import sentence_bleu
from sentence_transformers import SentenceTransformer, util
from tqdm.auto import tqdm
from transformers import AutoTokenizer, pipeline

import torch

import numpy as np
import pandas as pd

In [ ]:
df_a = pd.read_csv('./output/s2w_ko.csv')[['spoken_ko', 'written_ko']]
df_a.columns = ['spoken', 'written']
df_a

,spoken,written
0,사과는 잘 씻은 뒤 껍질 채 먹는 게 좋다네요.,사과 잘 씻어 골고루 먹는 게 좋다고 한다.
1,내가 언제까지 거기 가면 됩니까?,언제까지 갈까.
2,우리가 작성해서 전달해 드려야 하나요?,우리가 이것을 풀어서 전달해야 한다.
3,"난 지금 행복해요, 세상이 나를 원하고 있죠.","지금 행복하고, 세상이 나를 원한다."
4,그는 틀림없이 꽃 가게에 들렸을 겁니다.,꽃가게에 가봤어야 할 것 같다.
...,...,...
99995,그럼 컨셉을 약간 바꾸는 것도 좋은 방법이겠네요.,그러면 개념을 바꿔보는 게 좋지 않을까 하는 생각이 들었다.
99996,객실 안에 비밀번호가 작성되어 있습니다.,비밀번호는 방에 적혀있다.
99997,4번 게이트로 갔을 때 바로 찾을 수 있나요?,제가 4호 출입을 할 때 바로 찾을 수 있는 것이냐?
99998,지금 3개나 빠진 상태인데 한꺼번에 이렇게 많이 빠져도 상관없는 건가요?,이 중 3명이 빠져 있어 한꺼번에 다 빠져 나가는 게 말이 되냐는 것이다.


In [ ]:
'''def check_spelling(sent):
    spacing = Spacing()
    kospacing_sent = spacing(sent)
    hanspell_sent = spell_checker.check(sent).checked
    return hanspell_sent'''

'def check_spelling(sent):\n    spacing = Spacing()\n    kospacing_sent = spacing(sent)\n    hanspell_sent = spell_checker.check(sent).checked\n    return hanspell_sent'

In [ ]:
def filter(df):
    df['spoken'], df['written'] = df['spoken'].str.strip(), df['written'].str.strip()
    #df['written'] = df['written'].map(check_spelling)
    print(df.shape[0])
    
    # 1. 문장 길이를 이용한 필터링
    df = df.assign(
        spoken_length=df['spoken'].map(lambda x: len(x)),
        written_length=df['written'].map(lambda x: len(x))
    )

    mask = (df.spoken_length <= 100) & (df.written_length <= 100) & (df.written_length >= df.spoken_length * 0.9)
    display(df.loc[-mask, :])
    df = df.loc[mask, :]
    print(f'문장 길이를 이용한 필터링 -> {df.shape[0]}')

    # 2. BLEU score를 이용한 필터링
    okt = Okt()
    blue_score = []
    for idx, row in df.iterrows():
        candidate = okt.morphs(row['spoken'])
        references = [row['written']]
        references = [okt.morphs(s) for s in references]
        blue_score.append(sentence_bleu(references, candidate, weights=(1, 0, 0, 0)))
    df['blue_score'] = blue_score

    mask = (df.blue_score != 0) & (df.blue_score != 1)
    display(df.loc[-mask, :])
    df = df.loc[mask, :]
    print(f'BLEU score를 이용한 필터링 -> {df.shape[0]}')

    # 3. KLUE-NLI를 이용한 필터링
    classifier = pipeline(
        "text-classification",
        model="Huffon/klue-roberta-base-nli",
        return_all_scores=True,
        device=0
    )
    tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
    sep_token = tokenizer.sep_token
    nli_label = []
    nli_score = []
    df['concat'] = df['spoken'].str.cat(df['written'], sep=f' {sep_token} ')
    ds = Dataset.from_pandas(df)['concat']
    for output in tqdm(classifier(ds)):
        output_dict = {i['label']: i['score'] for i in output}
        label = max(output_dict.keys(), key=lambda x: output_dict[x])
        score = max(output_dict.values())
        nli_label.append(label)
        nli_score.append(score)
    df['nli_label'] = nli_label
    df['nli_score'] = nli_score
    df = df.drop(['concat'], axis=1)

    mask = (df.nli_label == 'ENTAILMENT') & (df.nli_score > 0.8)
    display(df.loc[-mask, :])
    df = df.loc[mask, :]
    print(f'KLUE-NLI를 이용한 필터링를 이용한 필터링 -> {df.shape[0]}')

    # 4. 코사인 유사도를 이용한 필터링
    model = SentenceTransformer('ddobokki/klue-roberta-small-nli-sts')
    model.to(torch.device('cuda'))
    df['combine'] = df['spoken'].map(lambda x: [x]) + df['written'].map(lambda x: [x])
    ds = Dataset.from_pandas(df)['combine']
    cosine_scores = []
    for sentences in tqdm(ds):
        embeddings = model.encode(sentences)
        #Compute cosine-similarities
        cosine_scores.append(util.cos_sim(embeddings[0], embeddings[1]))
    df['cosine_scores'] = cosine_scores
    df = df.drop(['combine'], axis=1)

    mask = (df.cosine_scores >= 0.9)
    display(df.loc[-mask, :])
    df = df.loc[mask, :]
    print(f'코사인 유사도를 이용한 필터링 -> {df.shape[0]}')

    df = df.reset_index(drop=True)
    return df

In [ ]:
df_b = df_a.head(10)
filter(df_b)

10


<ipython-input-14-264d12fc9d02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['spoken'], df['written'] = df['spoken'].str.strip(), df['written'].str.strip()


,spoken,written,spoken_length,written_length
1,내가 언제까지 거기 가면 됩니까?,언제까지 갈까.,18,8
3,"난 지금 행복해요, 세상이 나를 원하고 있죠.","지금 행복하고, 세상이 나를 원한다.",25,20
4,그는 틀림없이 꽃 가게에 들렸을 겁니다.,꽃가게에 가봤어야 할 것 같다.,22,17
5,"그녀가 좋아하는 가수는 박효신인데, 나는 그녀가 그와 같은 멋진 남자친구가 생기길 ...",그녀가 좋아하는 가수는 박효연이고 그녀가 좋은 가수를 뒀으면 좋겠다.,50,38
8,"아래 그림의 bga 부분에 대해서는, nsmd에서 smd로 존재하는 것과 같은 방식...",아래 사진의 BFM 부분 역시 변경이 있을 예정이다.,63,29


문장 길이를 이용한 필터링 -> 5


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

...

KLUE-NLI를 이용한 필터링를 이용한 필터링 -> 3


  0%|          | 0/3 [00:00<?, ?it/s]

,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
0,사과는 잘 씻은 뒤 껍질 채 먹는 게 좋다네요.,사과 잘 씻어 골고루 먹는 게 좋다고 한다.,26,24,0.454545,ENTAILMENT,0.923749,[[tensor(0.7967)]]
9,500달러만 당신들에게 요청할게요.,제가 여러분들께 500달러를 달라고 한다.,19,23,0.257733,ENTAILMENT,0.997867,[[tensor(0.7186)]]


코사인 유사도를 이용한 필터링 -> 1


,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
0,이 호텔은 세계 최악의 호텔로 유명합니다.,이 호텔은 세계에서 최악의 호텔로 유명하다.,23,24,0.814354,ENTAILMENT,0.998592,[[tensor(0.9792)]]


In [ ]:
# 4 * 100000/10 / 60 / 60

In [ ]:
df_a = filter(df_a)
df_a

100000


,spoken,written,spoken_length,written_length
1,내가 언제까지 거기 가면 됩니까?,언제까지 갈까.,18,8
3,"난 지금 행복해요, 세상이 나를 원하고 있죠.","지금 행복하고, 세상이 나를 원한다.",25,20
4,그는 틀림없이 꽃 가게에 들렸을 겁니다.,꽃가게에 가봤어야 할 것 같다.,22,17
5,"그녀가 좋아하는 가수는 박효신인데, 나는 그녀가 그와 같은 멋진 남자친구가 생기길 ...",그녀가 좋아하는 가수는 박효연이고 그녀가 좋은 가수를 뒀으면 좋겠다.,50,38
8,"아래 그림의 bga 부분에 대해서는, nsmd에서 smd로 존재하는 것과 같은 방식...",아래 사진의 BFM 부분 역시 변경이 있을 예정이다.,63,29
...,...,...,...,...
99982,"네, 저희 제품을 언제 구매하셨는지 알 수 있을까요?",너는 우리 제품을 언제 샀을까.,29,17
99988,"에, 채식은 미용상의 목적뿐만 아니라 위 건강의 나쁜 한국인들에게 적합한 식이요법의...",채식주의란 아름다움뿐만 아니라 건강한 한국인들의 건강을 위해 하는 것이다.,53,41
99994,"관람 가능한 표를 가지고 있는데, 편할 때 써도 된다고 쓰여있는데 무슨 말인가요?",관람권은 가지고 있지만 편리할 때 이용할 수 있다는 것이 안내문이다.,45,38
99996,객실 안에 비밀번호가 작성되어 있습니다.,비밀번호는 방에 적혀있다.,22,14


문장 길이를 이용한 필터링 -> 60499


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

,spoken,written,spoken_length,written_length,blue_score
34,밥 먹고 양치질은 언제 하세요?,식사 후에 두피를 닦아낼 때 : 뭘 해도 괜찮아.,17,27,0.0
342,그 파일 체크 아직도 다 안됐어?,아직 라인이 끝나지 않은 건가.,18,17,0.0
586,"실례지만, 길 좀 가르쳐 주시겠습니까?",제가 왜 그러냐고 하지만 저는 어떻게 하면 저희가 갈 수 있겠냐고 할 수 있다.,21,44,0.0
1000,빈티지한 분위기에서 슬로우 푸드를 맛볼 수 있다.,빈티지한 분위기에서 슬로우 푸드를 맛볼 수 있다.,27,27,1.0
1253,당신은 답신할 수 없을 정도로 바쁩니까?,여러분이 너무 바쁘셔서 말씀을 못 드리겠다.,22,24,0.0
...,...,...,...,...,...
99365,입을 때 아주 번거롭겠는데요?,반드시 입어줘야 하는데 너무 불편하다.,16,21,0.0
99520,"실례지만, 전원선 연결하셨나요?",제가 그걸 핑계해 드렸는데 그 연결을 해드렸습니다.,17,28,0.0
99603,저도 지나가다 본적이 있는데 조금 더 찾아보는 게 어떨까요?,그 전에도 봤는데 왜 그리 많이 그리냐고 하지 않으신다.,33,31,0.0
99653,그래서 그런지 몰라도 저 휴대전화 너무 좋아 보이지 않니?,그게 뭐냐 하면 전화기 잘 안 좋지만 전화기가 그렇게 좋은 것이 아니다.,32,40,0.0


...

KLUE-NLI를 이용한 필터링를 이용한 필터링 -> 29849


  0%|          | 0/29849 [00:00<?, ?it/s]

,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
0,사과는 잘 씻은 뒤 껍질 채 먹는 게 좋다네요.,사과 잘 씻어 골고루 먹는 게 좋다고 한다.,26,24,0.454545,ENTAILMENT,0.923749,[[tensor(0.7967)]]
9,500달러만 당신들에게 요청할게요.,제가 여러분들께 500달러를 달라고 한다.,19,23,0.257733,ENTAILMENT,0.997867,[[tensor(0.7186)]]
10,내년 오년동안의 창고 수용력 요구사항이 뭔가요?,내년에 보관장치(보관능력) 요구조건이 무엇일까.,26,26,0.197862,ENTAILMENT,0.996270,[[tensor(0.6253)]]
12,일본에 배 타고 가는 경우 몇 시간이 걸리나요?,배를 타고 일본으로 가는 길이 얼마나 갈까.,26,24,0.363636,ENTAILMENT,0.998040,[[tensor(0.8355)]]
13,다행이지만 많은 사람들이 죽어서 마음이 아파.,그래서 다행이지만 많은 분들이 돌아가셔서 안타까운 마음이 있다.,25,35,0.606366,ENTAILMENT,0.998653,[[tensor(0.8290)]]
...,...,...,...,...,...,...,...,...
99985,근데 그건 너무 흔한 이미지라서 좀 식상하네요.,다만 일반적인 이미지로 보기엔 무리가 있다.,26,24,0.159229,ENTAILMENT,0.998483,[[tensor(0.4600)]]
99991,제 숙소에 외부인이 들어온 거 같아요.,저는 제 숙소에 누군가가 오셨다고 생각합니다.,21,25,0.361935,ENTAILMENT,0.998660,[[tensor(0.6175)]]
99992,"그렇군요, 다음부터는 주문할 때 미리 말씀해주세요.",제가 보니까 다음 번을 주문할 때 미리 말씀드린다.,28,28,0.454774,ENTAILMENT,0.991985,[[tensor(0.8269)]]
99995,그럼 컨셉을 약간 바꾸는 것도 좋은 방법이겠네요.,그러면 개념을 바꿔보는 게 좋지 않을까 하는 생각이 들었다.,27,33,0.151591,ENTAILMENT,0.998520,[[tensor(0.6881)]]


코사인 유사도를 이용한 필터링 -> 6088


,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
0,이 호텔은 세계 최악의 호텔로 유명합니다.,이 호텔은 세계에서 최악의 호텔로 유명하다.,23,24,0.814354,ENTAILMENT,0.998592,[[tensor(0.9792)]]
1,미국이나 영국에서는 자신을 직접 소개하는 것은 바람직하지 않습니다.,미국이나 영국에서 자기 자신을 직접 소개하는 것이 바람직하지 않다.,37,37,0.731549,ENTAILMENT,0.998670,[[tensor(0.9765)]]
2,바쁘신데 긴 글 읽어주셔서 정말 감사해요.,바쁜데도 긴 글을 읽으셔서 감사드린다.,23,21,0.375000,ENTAILMENT,0.998609,[[tensor(0.9395)]]
3,남자는 수영장을 만들기로 하였습니다.,이 남성은 수영장을 만들기로 했다.,20,19,0.625000,ENTAILMENT,0.998424,[[tensor(0.9623)]]
4,그는 배우가 아닌 가수로 살아왔어.,그는 배우가 아닌 가수로 살아왔다.,19,19,0.888889,ENTAILMENT,0.998661,[[tensor(0.9951)]]
...,...,...,...,...,...,...,...,...
6083,그럼 마네킹이 입고있는 코트로 한번 입어볼 수 있을까요?,그렇다면 마네킹이 입었던 코트를 입어볼 수 있을까.,31,28,0.454545,ENTAILMENT,0.998500,[[tensor(0.9365)]]
6084,그런 경우는 거의 없지만 문제가 생기면 직원에게 물어보세요.,그럴 일이 거의 없는데 문제가 있으면 직원에게 물어본다.,33,31,0.461538,ENTAILMENT,0.997926,[[tensor(0.9236)]]
6085,"여보세요, 지금 피자 주문하면 배달 얼마나 걸리나요?",지금 피자를 주문하면 배달까지 얼마나 시간이 걸릴까.,29,29,0.491238,ENTAILMENT,0.998691,[[tensor(0.9072)]]
6086,할인을 받을 방법이 있을까요?,할인받을 방법이 있는 것일까.,16,16,0.495359,ENTAILMENT,0.998271,[[tensor(0.9673)]]


In [ ]:
df_a.to_csv('output/df_spoken-written_b_raw.csv')

In [ ]:
def filter2(df):
    mask = df['spoken'].str.contains('^.*(니다.)$', regex=True, na=False)
    display(df[mask])
    df = df[-mask]

    mask = df['written'].str.contains('^.*(요.)$', regex=True, na=False)
    display(df[mask])
    df = df[-mask]

    return df

In [ ]:
df_a = filter2(df_a)

<ipython-input-19-5c1e4ff06b50>:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask = df['spoken'].str.contains('^.*(니다.)$', regex=True, na=False)


,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
0,이 호텔은 세계 최악의 호텔로 유명합니다.,이 호텔은 세계에서 최악의 호텔로 유명하다.,23,24,0.814354,ENTAILMENT,0.998592,[[tensor(0.9792)]]
1,미국이나 영국에서는 자신을 직접 소개하는 것은 바람직하지 않습니다.,미국이나 영국에서 자기 자신을 직접 소개하는 것이 바람직하지 않다.,37,37,0.731549,ENTAILMENT,0.998670,[[tensor(0.9765)]]
3,남자는 수영장을 만들기로 하였습니다.,이 남성은 수영장을 만들기로 했다.,20,19,0.625000,ENTAILMENT,0.998424,[[tensor(0.9623)]]
6,내가 존을 배신했습니다.,저는 존을 배신했습니다.,13,13,0.714286,ENTAILMENT,0.997732,[[tensor(0.9658)]]
10,사랑하는 우리 이모 다시 한번 생일 축하합니다.,사랑하는 이모님이 다시 한 번 생일 축하드립니다.,26,27,0.573112,ENTAILMENT,0.998603,[[tensor(0.9326)]]
...,...,...,...,...,...,...,...,...
6060,혹시 현금으로 결제할 수 있으시면 약 30% 할인을 받으실 수 있습니다.,현금으로 결제할 수 있다면 30% 할인된 가격으로 구매할 수 있다.,40,37,0.561304,ENTAILMENT,0.998607,[[tensor(0.9248)]]
6067,오전 9시부터 오후 7시까지 이용 가능합니다.,오전 9시부터 오후 7시까지 이용이 가능하다.,25,25,0.795413,ENTAILMENT,0.998630,[[tensor(0.9886)]]
6071,"저는 고기를 못 먹습니다, 채식주의자입니다.","고기를 못 먹고, 채식주의자인데 제가 채식주의자다.",24,28,0.491238,ENTAILMENT,0.998703,[[tensor(0.9403)]]
6077,저도 가끔 실수를 하므로 이해할 수 있습니다.,제가 가끔 실수를 해서 이해를 할 수 있습니다.,25,26,0.664073,ENTAILMENT,0.998698,[[tensor(0.9350)]]


<ipython-input-19-5c1e4ff06b50>:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask = df['written'].str.contains('^.*(요.)$', regex=True, na=False)


,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
495,당신은 컴퓨터에 영화를 다운로드했나요?,영화를 컴퓨터로 다운로드 받으신 거예요?,21,22,0.555556,ENTAILMENT,0.998287,[[tensor(0.9211)]]
1142,당신은 요리를 자주 하시나요?,여러분이 자주 요리를 하실까요?,16,17,0.495359,ENTAILMENT,0.997140,[[tensor(0.9121)]]
1422,부모들은 정말 자식들의 결정에 있어 큰 영향을 발휘하는가?,부모들이 자녀들의 결정에 큰 영향을 미치는 것은 정말인가요?,32,33,0.704560,ENTAILMENT,0.998378,[[tensor(0.9260)]]
1652,내용 및 공장 온도를 확인하세요.,내용 및 공장 온도 등을 확인해 주세요.,18,22,0.584101,ENTAILMENT,0.998617,[[tensor(0.9740)]]
1676,비가 많이 왔었는데 어제 어떻게 보냈어요?,비가 많이 내렸는데 어제 보내주셨나요?,23,21,0.625000,ENTAILMENT,0.998632,[[tensor(0.9091)]]
2179,너 인도네시아 검역 절차에 대해 알고 있어?,인도네시아의 검역 절차에 대해서 알고 계신가요?,24,26,0.723870,ENTAILMENT,0.979848,[[tensor(0.9544)]]
2653,어제 부모님께 내가 만든 간장새우 드렸어?,어제 제가 만든 간장을 부모님께 드렸더니 그거예요?,23,28,0.573112,ENTAILMENT,0.989094,[[tensor(0.9056)]]
2734,당신을 소개하기 위한 간단한 이력서를 준비해 주세요.,자신을 소개할 수 있는 간단한 이력서를 작성해 주세요.,29,30,0.615385,ENTAILMENT,0.997224,[[tensor(0.9184)]]
2880,제가 경기장으로 데려다 드릴테니 걱정 마세요.,제가 경기장에 데려다 드릴 테니 걱정 마세요.,25,25,0.622796,ENTAILMENT,0.998335,[[tensor(0.9843)]]
2941,당신은 정말로 저를 좋아하는 것입니까?,"제가 정말 좋아하는 건, 저를 좋아하시는 건가요?",21,27,0.318458,ENTAILMENT,0.968358,[[tensor(0.9082)]]


In [ ]:
df_a.to_csv('output/df_spoken-written_b_raw2.csv')

In [ ]:
df_a[['spoken', 'written']].to_csv('output/df_spoken-written_b.csv',  encoding = "utf-8-sig")

In [ ]:
pd.read_csv('output/df_spoken-written_b.csv')

,Unnamed: 0,spoken,written
0,2,바쁘신데 긴 글 읽어주셔서 정말 감사해요.,바쁜데도 긴 글을 읽으셔서 감사드린다.
1,4,그는 배우가 아닌 가수로 살아왔어.,그는 배우가 아닌 가수로 살아왔다.
2,5,디자이너들은 재활용제품을 친환경 제품으로 바꾸기 위해 사용해요.,디자이너들이 재활용품을 활용해 친환경 제품을 바꾸기도 한다.
3,7,갑자기 왜 이런 문제가 발생하는 건가요?,왜 갑자기 이런 문제가 생기는 건가 말이다.
4,8,이 의견에 동의하지 않는 분 계신가요?,이 의견에 동의하지 않는 사람이 있을까.
...,...,...,...
3650,6083,그럼 마네킹이 입고있는 코트로 한번 입어볼 수 있을까요?,그렇다면 마네킹이 입었던 코트를 입어볼 수 있을까.
3651,6084,그런 경우는 거의 없지만 문제가 생기면 직원에게 물어보세요.,그럴 일이 거의 없는데 문제가 있으면 직원에게 물어본다.
3652,6085,"여보세요, 지금 피자 주문하면 배달 얼마나 걸리나요?",지금 피자를 주문하면 배달까지 얼마나 시간이 걸릴까.
3653,6086,할인을 받을 방법이 있을까요?,할인받을 방법이 있는 것일까.


# References

- https://drive.google.com/file/d/14SnyQH0GUkPYHBuz52OLTwcjKmbsGlJF/view?usp=share_link
- http://incredible.ai/nlp/2020/02/29/BLEU/
- https://github.com/Huffon/klue-transformers-tutorial/blob/master/natural_language_inference.ipynb
- https://huggingface.co/ddobokki/klue-roberta-small-nli-sts
- https://www.sbert.net/docs/usage/semantic_textual_similarity.html